<a href="https://colab.research.google.com/github/luisarciniegas/IA-Udea/blob/main/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrega Final
En este entrega tendrás que someter tu solución a Kaggle. CADA MIEMBRO DE CADA GRUPO TIENE QUE SOMETER SU PROPIA SOLUCIÓN, aunque sea la misma que la del resto de componentes del grupo.

Además del contenido de las entregas anteriores tendrás que añadir a tu repositorio github

- README.md con los nombres, cédulas y progama en el que está inscrito cada participante y con los siguientes enlaces y con el enlace al video de la - entrega 2 y el enlace al video de esta entrega.

- **Video subido a YouTube y enlazado en README.md**

    - Elabora un vídeo de entre 3mins y 4mins.

    - En el video ha de explicar brevemente la aproximación a la solución enviada a Kaggle y ha de mostrar vuestra posición en el ranking compartiendo la pantalla mientras se consulta el leaderboard de Kaggle. Esta posición, por supuesto, puede que no sea la definitiva si hay equipos que envian sus soluciones después.

- 99 - modelo solución.ipynb - Un notebook con tu solución completa, desde cargar train.csv hasta generar el archivo que se envía a Kaggle. Mira las NOTES 05.03 para tener un ejemplo.

- al menos dos notebooks adicionales con aproximaciones distantas a la de la solución, porque usan distintos modelos, o estrategias de limpieza o preprocesado, etc.

Los criterios de evaluación son los siguientes:

- CLARIDAD: los notebooks contienen textos que explican brevemente lo que se está haciendo
- COMPLECIÓN: la entrega contiene todos los notebooks que se describieron justo arriba.
- REPRODUCIBILIDAD: la solución que genera el notebook 99 - modelo solución.ipynb coincide con la enviada a Kaggle.

Al final tu repositorio habrá de tener la siguiente estructura:  

    +-  README-md
    +-  01 - exploración.ipynb
    +-  02 - preprocesado.ipynb
    +-  03 - modelo con preprocesado de tal forma y SVM.ipynb
    +-  04 - modelo con preprocesado de otra forma y Random Forest.ipynb
    +-  99 - modelo solución.ipynb
    
El nombrado de los archivos ES ESTRICTO. Tiene que haber un fichero README.md y los notebooks 01, 02 y 99 nombrados EXACTAMENTE IGUAL que se indica arriba. Los notebooks 03, 04 y posteriores (si tienes más), han de ser nombrados con ese esquema (número consecutivo - nombre_que_escojes.ipynb). Cualquier archivo con otro nombre será ignorado.

Cada miembro del grupo ES RESPONSABLE DE SU PROPIA ENTREGA, independientemente de si sus compañeros de grupo entregan o no. Esto incluye rellenar el formulario y tener un github personal, aunque el contenido sea el mismo que el del resto de los compañeros de grupo.

Si cambias de grupo o te añades a un grupo que ya exista tienes que enviar un correo a raul.ramos@udea.edu.co, felipe.rodrigueza@udea.edu.co informándolo con copia a tus nuevos compañeros indicando las razones del cambio y cuál será tu rol en el nuevo grupo. Tus nuevos compañeros deberán de responder ese correo autorizandote a que formes parte del grupo.

In [1]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

replicating local resources


## Download data directly from Kaggle

- create a file `kaggle.json` with your authentication token (in kaggle $\to$ click user icon on top-right $\to$ settings $\to$ API create new token)
- upload it to this notebook workspace
- run the following cell

In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

 55% 11.0M/20.1M [00:00<00:00, 106MB/s]
100% 20.1M/20.1M [00:00<00:00, 130MB/s]


## unzip and inspect data

In [3]:
!unzip udea*.zip > /dev/null

In [4]:
# Importamos librerías
import os
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import (
    confusion_matrix,
    mean_absolute_error,
    accuracy_score,
    mean_squared_error,
    precision_score,
    f1_score,
    recall_score
)
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [5]:
# Lectura del archivo "train.csv"

nombre_archivo = "train.csv"

#Creamos el DataFrame
df = pd.read_csv(nombre_archivo)

# Mostrar un breve resumen del archivo "train.csv"
df.head(10)

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo
5,659872,20203,MEDICINA VETERINARIA,ANTIOQUIA,Más de 7 millones,Menos de 10 horas,Estrato 5,Si,Educación profesional completa,Secundaria (Bachillerato) completa,No,medio-alto
6,47159,20183,INGENIERIA MECANICA,HUILA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Educación profesional incompleta,Técnica o tecnológica completa,Si,alto
7,11829,20183,ADMINISTRACIÓN EN SALUD OCUPACIONAL,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,Entre 11 y 20 horas,Estrato 2,Si,Primaria incompleta,Secundaria (Bachillerato) incompleta,Si,medio-bajo
8,257869,20212,INGENIERIA INDUSTRIAL,ATLANTICO,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 1,Si,Secundaria (Bachillerato) completa,Educación profesional incompleta,Si,medio-bajo
9,465511,20183,ADMINISTRACION DE EMPRESAS,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 5,Si,Postgrado,Postgrado,Si,alto


In [6]:
#Información dimensiones archivo "train.csv":
print("\nTamaño del archivo:",
      "\nFilas:", df.shape[0],
      "\nColumnas:", df.shape[1])

#Verificar datos faltantes para la limpieza y el preprocesado
print("\nDatos Faltantes por columna:\n",df.isnull().sum())


Tamaño del archivo: 
Filas: 692500 
Columnas: 12

Datos Faltantes por columna:
 ID                                    0
PERIODO                               0
ESTU_PRGM_ACADEMICO                   0
ESTU_PRGM_DEPARTAMENTO                0
ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
FAMI_EDUCACIONMADRE               23664
ESTU_PAGOMATRICULAPROPIO           6498
RENDIMIENTO_GLOBAL                    0
dtype: int64


In [7]:
# Lectura del archivo "test.csv" para evaluar el modelo entrenado
nombre_archivo = "test.csv"

#Creamos el DataFrame
df_test = pd.read_csv(nombre_archivo)

# Mostrar un breve resumen del archivo "train.csv"
df_test.head(10)

,Unnamed: 0,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO
0,0,550236,20183,TRABAJO SOCIAL,BOLIVAR,Menos de 500 mil,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica completa,Primaria completa,Si
1,1,98545,20203,ADMINISTRACION COMERCIAL Y DE MERCADEO,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Técnica o tecnológica completa,No
2,2,499179,20212,INGENIERIA MECATRONICA,BOGOTÁ,Entre 1 millón y menos de 2.5 millones,0,Estrato 3,Si,Secundaria (Bachillerato) incompleta,Secundaria (Bachillerato) completa,No
3,3,782980,20195,CONTADURIA PUBLICA,SUCRE,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 1,No,Primaria incompleta,Primaria incompleta,No
4,4,785185,20212,ADMINISTRACION DE EMPRESAS,ATLANTICO,Entre 2.5 millones y menos de 4 millones,Entre 11 y 20 horas,Estrato 2,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No
5,5,58495,20183,ADMINISTRACION AGROPECUARIA,VALLE,Entre 500 mil y menos de 1 millón,Más de 30 horas,Estrato 1,Si,Ninguno,Secundaria (Bachillerato) completa,Si
6,6,705444,20195,INGENIERIA CIVIL,SANTANDER,Menos de 500 mil,0,Estrato 3,Si,Primaria completa,Técnica o tecnológica completa,No
7,7,557548,20183,INGENIERÍA MECÁNICA,BOGOTÁ,Más de 7 millones,Menos de 10 horas,Estrato 4,Si,Postgrado,Postgrado,No
8,8,519909,20195,INGENIERÍA DE SISTEMAS,ATLANTICO,Entre 1 millón y menos de 2.5 millones,Entre 21 y 30 horas,Estrato 2,No,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No
9,9,832058,20183,ECONOMIA,BOGOTÁ,Entre 500 mil y menos de 1 millón,Entre 21 y 30 horas,Estrato 2,Si,Técnica o tecnológica completa,Secundaria (Bachillerato) incompleta,No


# Detalles del código propuesto:

Este código realiza dos tareas principales:

**Preprocesamiento de Datos (preprocess_data):**
- Elimina columnas no relevantes y separa las características (X) de las etiquetas (y).
- Las características categóricas son procesadas con OneHotEncoder y las ordinales con OrdinalEncoder y StandardScaler.
- Devuelve las características preprocesadas y las etiquetas.

**Entrenamiento del Modelo Random Forest (rf_classifier):**
- Utiliza StratifiedKFold para validación cruzada estratificada.
- Entrena un modelo de Random Forest con varios hiperparámetros como max_depth, min_samples_split, y n_estimators.
- Calcula métricas de rendimiento como accuracy, precision, recall y F1-score para cada pliegue.
- Registra los resultados en un DataFrame y devuelve las métricas y la matriz de confusión acumulada.

# Función para el preprocesado de datos de entrenamiento

In [8]:
# Preprocesamiento de los datos (entrenamiento y prueba)
def preprocess_data(df, is_training=True, preprocessor=None):
    # Separar las características (X) y la variable objetivo (y)
    X = df.drop(columns=['ID', 'RENDIMIENTO_GLOBAL'], errors='ignore')  # 'errors="ignore"' para que no lance error si no existe 'RENDIMIENTO_GLOBAL'
    y = df.get('RENDIMIENTO_GLOBAL', None)  # Solo 'y' si existe la columna

    # Características categóricas y ordinales
    categorical_features = [
        'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
        'FAMI_TIENEINTERNET', 'ESTU_PAGOMATRICULAPROPIO'
    ]
    ordinal_features = [
        'ESTU_VALORMATRICULAUNIVERSIDAD', 'FAMI_ESTRATOVIVIENDA',
        'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE',
        'ESTU_HORASSEMANATRABAJA'
    ]

    # Preprocesamiento para características categóricas y ordinales
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    ordinal_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('ordinal', OrdinalEncoder()),
        ('scaler', StandardScaler())
    ])

    # Combinamos todo en un preprocesador
    if preprocessor is None:  # Si es la primera vez que se entrena
        preprocessor = ColumnTransformer(
            transformers=[
                ('cat', categorical_transformer, categorical_features),
                ('ord', ordinal_transformer, ordinal_features)
            ])

    # Ajustar o transformar según si estamos entrenando o no
    if is_training:
        X_preprocessed = preprocessor.fit_transform(X)  # Ajuste y transformación para los datos de entrenamiento
    else:
        X_preprocessed = preprocessor.transform(X)  # Solo transformación para los datos de prueba

    return X_preprocessed, y, preprocessor

# Mejor modelo obtenido de Random Forest


In [21]:

# Función para entrenar el modelo Random Forest
def rf_classifier(X, y, num_trees):
    # Convierte y a etiquetas categóricas
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # Definir StratifiedKFold
    skf = StratifiedKFold(n_splits=4)

    resultados = pd.DataFrame()
    conf_matrix_list_of_arrays = np.zeros((len(np.unique(y_encoded)), len(np.unique(y_encoded))))

    idx = 0
    for trees in num_trees:
        EficienciaTrain = []
        EficienciaVal = []
        Macc = []
        Mpre = []
        Mrec = []
        Mf1 = []

        print(f"Training with Random Forest (n_estimators={trees})...")

        for train_index, test_index in skf.split(X, y_encoded):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y_encoded[train_index], y_encoded[test_index]

            # Modelo Random Forest Optimizado
            model = RandomForestClassifier(
                n_estimators=trees,
                random_state=0,
                class_weight='balanced',
                max_depth=20,
                min_samples_split=20,
                min_samples_leaf=10,
                max_features='sqrt',
                n_jobs=-1
            )

            model.fit(X_train, y_train)

            y_pred = model.predict(X_test)
            y_pred_train = model.predict(X_train)

            conf_matrix = confusion_matrix(y_test, y_pred)
            conf_matrix_list_of_arrays += conf_matrix

            EficienciaTrain.append(np.mean(y_pred_train.ravel() == y_train.ravel()))
            EficienciaVal.append(np.mean(y_pred.ravel() == y_test.ravel()))
            Macc.append(accuracy_score(y_test, y_pred))
            Mpre.append(precision_score(y_test, y_pred, average="weighted"))
            Mrec.append(recall_score(y_test, y_pred, average="weighted"))
            Mf1.append(f1_score(y_test, y_pred, average="weighted"))

        # Resultados por parámetro
        resultados.loc[idx, 'n_estimators (número de árboles)'] = trees
        resultados.loc[idx, 'eficiencia de entrenamiento'] = np.mean(EficienciaTrain)
        resultados.loc[idx, 'desviación estándar entrenamiento'] = np.std(EficienciaTrain)
        resultados.loc[idx, 'eficiencia de prueba'] = np.mean(EficienciaVal)
        resultados.loc[idx, 'Intervalo de confianza (prueba)'] = np.std(EficienciaVal)
        resultados.loc[idx, 'accuracy real'] = np.mean(Macc)
        resultados.loc[idx, 'precision_score'] = np.mean(Mpre)
        resultados.loc[idx, 'recall_score'] = np.mean(Mrec)
        resultados.loc[idx, 'f1_score'] = np.mean(Mf1)
        idx += 1

    return resultados, conf_matrix_list_of_arrays, model  # Devuelve el último modelo entrenado

In [22]:
# Preprocesar datos de entrenamiento
X_train, y_train, preprocessor = preprocess_data(df, is_training=True)

# Entrenar el modelo Random Forest
resultados, conf_matrix, best_model = rf_classifier(X_train, y_train, num_trees=[100])

Training with Random Forest (n_estimators=100)...


In [33]:
resultados

,n_estimators (número de árboles),eficiencia de entrenamiento,desviación estándar entrenamiento,eficiencia de prueba,Intervalo de confianza (prueba),accuracy real,precision_score,recall_score,f1_score
0,100.0,0.413914,0.000706,0.400438,0.00049,0.400438,0.376427,0.400438,0.375819


In [45]:
# Preprocesar datos de prueba
X_test, y_test, _ = preprocess_data(df_test, is_training=False, preprocessor=preprocessor)

# Predicción con el modelo entrenado
predicciones = best_model.predict(X_test)

In [46]:
X_test

<296786x988 sparse matrix of type '<class 'numpy.float64'>'
	with 2671064 stored elements in Compressed Sparse Row format>

In [47]:
# Crear un DataFrame con las predicciones
df_predicciones = pd.DataFrame(predicciones, columns=['RENDIMIENTO_GLOBAL_PREDICTED'])

# Si deseas agregar las predicciones al archivo de test original:
df_test['RENDIMIENTO_GLOBAL_PREDICTED'] = df_predicciones

# Guardar el archivo con las predicciones
df_test.to_csv('test_with_predictions.csv', index=False)

# Mostrar las primeras predicciones
print(df_predicciones.head())


   RENDIMIENTO_GLOBAL_PREDICTED
0                             1
1                             2
2                             0
3                             1
4                             1


In [39]:
len(predicciones)

296786

In [40]:
df_submission = pd.DataFrame({"ID": df_test["ID"], "RENDIMIENTO_GLOBAL": predicciones})

In [41]:
# Mapeo de valores numéricos a las categorías de texto
label_mapping = {
    0: 'bajo',
    1: 'medio-bajo',
    2: 'medio-alto',
    3: 'alto'
}

# Asignar las categorías correspondientes a las predicciones numéricas
predicciones_texto = [label_mapping[pred] for pred in predicciones]

# Crear un DataFrame con las predicciones en formato texto
df_submission = pd.DataFrame({
    'ID': df_test['ID'],  # Asumiendo que 'df_test' contiene las IDs correspondientes
    'RENDIMIENTO_GLOBAL': predicciones_texto # Las predicciones en formato texto
})

# Guardar el archivo de envío en formato CSV
df_submission.to_csv("submission.csv", index=False)
print("Archivo 'submission.csv' creado con éxito.")

df_submission.shape

!head submission_optimized.csv

Archivo 'submission.csv' creado con éxito.
head: cannot open 'submission_optimized.csv' for reading: No such file or directory


In [42]:
!head submission.csv

ID,RENDIMIENTO_GLOBAL
550236,medio-bajo
98545,medio-alto
499179,bajo
782980,medio-bajo
785185,medio-bajo
58495,medio-bajo
705444,bajo
557548,bajo
519909,medio-bajo


In [43]:
# Enviar archivo competencia
!kaggle competitions submit -c udea-ai4eng-20242 -f submission.csv -m "Luisa Carolina Rodríguez Arciniegas solución con Random Forest"

100% 4.21M/4.21M [00:00<00:00, 14.7MB/s]
Successfully submitted to UDEA/ai4eng 20242 - Pruebas Saber Pro Colombia